In [20]:
import json
import pickle
from tensorflow.keras.models import load_model
import nltk
import numpy as np
import torch
import random
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [21]:
# Load data json
with open('intents.json') as content:
    data = json.load(content)

# Load words 
words = pickle.load(open('words.pkl', 'rb'))

# Load classes
classes = pickle.load(open('classes.pkl', 'rb'))

# Load model
model = load_model('D:/Project_Data/project/Project Pribadi/Chatbot Website/Code baru/chatbot_model') 

In [22]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [23]:
def clean_up_classes(sentence):
    """
    Tokenizes and stems the input sentence using Sastrawi stemmer.

    Args:
    sentence (str): Input sentence to be tokenized and stemmed.

    Returns:
    list: List of tokenized and stemmed words.
    """
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    """
    Converts a sentence into a bag-of-words representation based on the loaded vocabulary.

    Parameters:
    ----------
    sentence : str
        Input sentence to be converted.

    Returns:
    --------
    numpy.ndarray
        Bag-of-words representation of the input sentence.
    """
    # Tokenize and stem the input sentence
    sentence_words = clean_up_classes(sentence)
    
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    """
    Predicts the intent class of the input sentence using the loaded model.

    Parameters:
    ----------
    sentence : str 
        Input sentence for intent prediction.

    Returns:
    --------
    list: 
        List of dictionaries containing the predicted intent and its probability.
    """
    sentence_words = clean_up_classes(sentence)

    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    """
    Predicts the intent class of the input sentence using the loaded model.

    Parameters:
    ----------
    sentence : str 
        Input sentence for intent prediction.

    Returns:
    --------
    list: 
        List of dictionaries containing the predicted intent and its probability.
    """
    # Convert the sentence into a bag-of-words representation
    bow = bag_of_words(sentence)

    # Make a prediction using the loaded model
    res = model.predict(np.array([bow]))[0]
    
    # Apply a threshold to filter out predictions with low probabilities
    ERROR_THRESHOLD = 0.25
    
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    
    return return_list

In [24]:
def get_response(intents_list, intents_json):
    """
    Retrieves a random response for the predicted intent from the loaded data.

    Parameters:
    -----------
    intents_list : list 
        List of dictionaries containing the predicted intent and its probability.
    intents_json : dict 
        Dictionary containing the intent data loaded from the JSON file.

    Returns:
    --------
    str: 
        Randomly selected response for the predicted intent.
    """
    # If intents_list is empty, set a default response
    if not intents_list:
        return "Maaf, saya tidak mengerti pertanyaan Anda."

    # Extract the predicted intent
    tag = intents_list[0]['intent']
    
    # Find the intent with matching tag
    matching_intent = next((intent for intent in intents_json['intents'] if intent['tag'] == tag), None)

    # If matching intent is found, choose a random response
    if matching_intent:
        result = random.choice(matching_intent['responses'])
    else:
        # If no matching intent is found, set a default response
        result = "Maaf, saya tidak mengerti pertanyaan Anda."

    return result

In [25]:
print("Hallo Bot Zen akan membantu Anda")

while True:
    message = input("You: ")
    
    # Check if the user wants to exit
    if message.lower() == 'exit':
        print("Bot: Goodbye!")
        break
    
    ints = predict_class(message)
    res = get_response(ints, data)
    print("Bot:", res)

Hallo Bot Zen akan membantu Anda
1/1 [==============================] - 0s 52ms/step
Bot: Hi Tuan
1/1 [==============================] - 0s 23ms/step
Bot: PT. Zenithera Kreasi adalah startup riset dan konsultasi yang berfokus pada data dan teknologi luas. Didirikan sebagai ZENITH pada tahun 2023, kami memiliki beberapa layanan business-to-business (B2B) business to Government (B2G), di bidang: Business dan Data
1/1 [==============================] - 0s 21ms/step
Bot: Maaf, saya tidak mengerti pertanyaan Anda.
Bot: Goodbye!
